<a href="https://colab.research.google.com/github/sidrusiya/AILA_Tasks/blob/main/AILA_Task2_L1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Processed Data**

In [32]:
import numpy as np
import pandas as pd
import os
import nltk
import re
import json

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import seaborn as sns
import gensim
nltk.download('all')
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from google.colab import drive
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
!pip3 install tensorflow_text
import tensorflow_text
!pip install -q tf-models-official
!pip install -q -U tensorflow-text
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import tensorflow_hub as hub
import tensorflow_text as tf_text

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [35]:
dataset = open('/content/drive/MyDrive/nit_agartala_nlp_team_3.tsv', 'r')
df = pd.read_csv(dataset, sep='\t')

In [36]:
df

,judge_id,id,sentence,relevance,lem_text,stem_text
0,1,1,Appeal by special leave from the order dated ...,0,appeal special leave order dated government pu...,appeal special leav order date govern punjab (...
1,2,1,From the Judgment and Order dated . . of the ...,0,judgment order dated . . bombay high court app...,judgment order date . . bombay high court appe...
2,3,1,Appeal from the judgment and decree dated rd/t...,0,appeal judgment decree dated rd/th calcutta hi...,appeal judgment decre date rd/th calcutta high...
3,4,1,of .,0,.,.
4,5,1,From the Judgment and order dated . . of the ...,0,judgment order dated . . allahabad high court ...,judgment order date . . allahabad high court c...
...,...,...,...,...,...,...
72187,27,2541,The petitioner before us does not complain tha...,0,petitioner u complain got proper grounds.,petition us complain got proper grounds.
72188,27,2542,Further the period of his detention under the ...,0,period detention impugned act gone beyond thre...,period detent impugn act gone beyond three mon...
72189,27,2543,Petition dismissed.,0,petition dismissed.,petit dismissed.
72190,27,2544,Agent for the petitioner: S. Subrahmanyam.,0,agent petitioner: s. subrahmanyam.,agent petitioner: s. subrahmanyam.


# **Balancing Imbalanced Dataset**

In [37]:
df.relevance.value_counts()

0    55611
1    16581
Name: relevance, dtype: int64

In [38]:
from sklearn.utils import shuffle

lemm = []
relevance = []
THRESH_1 = 40000

for group in df.relevance.unique():
    if len(df.groupby('relevance').get_group(group).lem_text.unique()) > THRESH_1:
        lemm += (list(shuffle(df.groupby('relevance').get_group(group).lem_text.unique()))[:THRESH_1])
        relevance += ([group]*THRESH_1)

    else:
        lemm += (list(shuffle(df.groupby('relevance').get_group(group).lem_text.unique())))
        relevance += ([group]*len(list(shuffle(df.groupby('relevance').get_group(group).lem_text.unique()))))

In [39]:
len(lemm)

56399

In [40]:
len(relevance)

56399

In [41]:
balanced_df = pd.DataFrame({'Text' : lemm, 'Relevance': relevance})
balanced_df

,Text,Relevance
0,taking consideration representation made peopl...,0
1,say state shall make law take away abridges ri...,0
2,whether order passed sikandar bakht minister w...,0
3,matter least two lady happen estate death quit...,0
4,trial court rejected case plaintiff adopted wi...,0
...,...,...
56394,supreme court reporter .,1
56395,tribunal statement present appeal however stat...,1
56396,opinion learned civil judge error view provisi...,1
56397,notice issued niil's secretary intimating meet...,1


# **BERT**

In [42]:
max_classes = len(balanced_df.Relevance.unique())

In [43]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
result = shuffle(balanced_df)

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(balanced_df.Text, balanced_df.Relevance, 
                                                    test_size = 0.1, random_state = 42, stratify =balanced_df.Relevance)

In [45]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'
bert_model_link = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
bert_model_preprocessing_link = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [46]:
bert_preprocess_model = hub.KerasLayer(bert_model_preprocessing_link)

In [47]:
bert_model = hub.KerasLayer(bert_model_link)

In [48]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype = tf.string, name='input-text')
    preprocessing_layer = hub.KerasLayer(bert_model_preprocessing_link, name='bert-preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(bert_model_link, trainable=True, name='bert-encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = Dense(units= 512, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    net = Dense(units=max_classes, activation='softmax', name='classifer')(net)
    return tf.keras.Model(text_input, net)

In [49]:
classifier_model = build_classifier_model()
# bert_raw_result = classifier_model(tf.constant(sample_text))

In [50]:
classifier_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input-text (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 bert-preprocessing (KerasLayer  {'input_mask': (Non  0          ['input-text[0][0]']             
 )                              e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [51]:
%time
encoded_labels_train = to_categorical(Y_train, num_classes=max_classes)
encoded_labels_test = to_categorical(Y_test, num_classes=max_classes)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 23.1 µs


In [52]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

EPOCHS = 3
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
batch_size = 32

num_train_steps =  len((df.relevance) // batch_size) * EPOCHS
lr_scheduler = PolynomialDecay(initial_learning_rate=5e-5,
                               end_learning_rate=0.,
                               decay_steps=num_train_steps
                               )

from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=lr_scheduler)

In [53]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
!mkdir checkpoints

MyCheckpoint = ModelCheckpoint('/content/checkpoints',
                               monitor = 'val_categorical_accuracy',
                               verbose = 1,
                               save_best_only = True,
                               save_weights_only = False)

MyEarlyStopping = EarlyStopping(patience =1,
                                monitor='val_categorical_accuracy',
                                restore_best_weights = True,
                                verbose = 1)

import os
os.path.exists('/content/checkpoints')

mkdir: cannot create directory ‘checkpoints’: File exists


True

In [54]:
from sklearn.utils import class_weight
class_weights_array = class_weight.compute_class_weight(class_weight='balanced'
                                               ,classes=np.unique(balanced_df.Relevance)
                                               ,y=balanced_df.Relevance)

class_weights_array

array([0.7049875 , 1.71958656])

In [55]:
class_weights = {i : class_weights_array[i] for i in range(len(class_weights_array))}
class_weights

{0: 0.7049875, 1: 1.7195865601561071}

In [56]:
import tensorflow_addons as tfa

macro_f1 = tfa.metrics.F1Score(num_classes=max_classes, average='macro')
micro_f1 = tfa.metrics.F1Score(num_classes=max_classes, average='micro')
weighted_f1 = tfa.metrics.F1Score(num_classes=max_classes, average='weighted')
precision = tf.metrics.Precision()
recall = tf.metrics.Recall()
categorical_accuracry = tf.metrics.CategoricalAccuracy()

metrics = [categorical_accuracry, macro_f1, precision, recall]

In [57]:
classifier_model.compile(optimizer= opt,
              loss= loss,
              metrics = metrics)

In [58]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, encoded_labels_train)).shuffle(70000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, encoded_labels_test)).shuffle(70000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [59]:
classifier_model.fit(train_dataset,
                     validation_data=test_dataset,
                 epochs = EPOCHS,
                 verbose = 1,
                 class_weight=class_weights,
                 callbacks = [MyCheckpoint, MyEarlyStopping]
          )

Epoch 1/3
1587/1587 [==============================] - ETA: 0s - loss: 0.6556 - categorical_accuracy: 0.5897 - f1_score: 0.5707 - precision_1: 0.5897 - recall_1: 0.5897
Epoch 00001: val_categorical_accuracy improved from -inf to 0.68901, saving model to /content/checkpoints


INFO:tensorflow:Assets written to: /content/checkpoints/assets


INFO:tensorflow:Assets written to: /content/checkpoints/assets


1587/1587 [==============================] - 2665s 2s/step - loss: 0.6556 - categorical_accuracy: 0.5897 - f1_score: 0.5707 - precision_1: 0.5897 - recall_1: 0.5897 - val_loss: 0.5702 - val_categorical_accuracy: 0.6890 - val_f1_score: 0.6382 - val_precision_1: 0.6890 - val_recall_1: 0.6890
Epoch 2/3
1587/1587 [==============================] - ETA: 0s - loss: 0.5818 - categorical_accuracy: 0.6762 - f1_score: 0.6551 - precision_1: 0.6762 - recall_1: 0.6762
Epoch 00002: val_categorical_accuracy did not improve from 0.68901
Restoring model weights from the end of the best epoch: 1.
1587/1587 [==============================] - 2607s 2s/step - loss: 0.5818 - categorical_accuracy: 0.6762 - f1_score: 0.6551 - precision_1: 0.6762 - recall_1: 0.6762 - val_loss: 0.6976 - val_categorical_accuracy: 0.6032 - val_f1_score: 0.5944 - val_precision_1: 0.6032 - val_recall_1: 0.6032
Epoch 00002: early stopping


In [60]:
classifier_model.save('BERT-Model.h5') 

In [79]:
from google.colab import drive
drive.mount('/content/drive')
dataset1 = open('/content/drive/MyDrive/Task2_Test/aila_test_judgement.json', 'r')
data1 = json.load(dataset1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
yz1 = pd.DataFrame.from_dict(data1, orient='index')
yz1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
1,"{'id': '1', 'sentence': 'From the Judgment and...","{'id': '2', 'sentence': 'W. P. numbers 2274, ...","{'id': '3', 'sentence': 'Rajinder Sachher and ...","{'id': '4', 'sentence': 'Harbans Lal, Dr. Mee...","{'id': '5', 'sentence': 'The Judgment of the C...","{'id': '6', 'sentence': 'These five appeals by...","{'id': '7', 'sentence': 'Writ Petition number ...","{'id': '8', 'sentence': 'Writ Peti tion number...","{'id': '9', 'sentence': 'Since identical conte...","{'id': '10', 'sentence': 'As it is said that C...","{'id': '11', 'sentence': 'Writ Petition number...","{'id': '12', 'sentence': 'The Bareilly Develop...","{'id': '13', 'sentence': 'as 'BDA'), the first...","{'id': '14', 'sentence': ', With a view to cas...","{'id': '15', 'sentence': 'The BDA issued' an a...","{'id': '16', 'sentence': 'In this appeal i. e....","{'id': '17', 'sentence': 'The following table ...","{'id': '18', 'sentence': 'HIG rupees 1500 and ...","{'id': '19', 'sentence': 'LIG rupees 351 to ru...","{'id': '20', 'sentence': 'EWS rupees 350 p. m....","{'id': '21', 'sentence': 'The note under the '...","{'id': '22', 'sentence': 'All the respondents ...","{'id': '23', 'sentence': 'their names' for MIG...","{'id': '24', 'sentence': 'Thereafter, the resp...","{'id': '25', 'sentence': '1. 84 from the Secre...","{'id': '26', 'sentence': 'Cost of the house ru...","{'id': '27', 'sentence': 'Down payment to be m...","{'id': '28', 'sentence': 'Rate of yearly inter...","{'id': '29', 'sentence': 'Amount of monthly in...","{'id': '30', 'sentence': '50 with interest.', ...","{'id': '31', 'sentence': 'By the said notice, ...","{'id': '32', 'sentence': 'Except the responden...","{'id': '33', 'sentence': 'Hence their names we...","{'id': '34', 'sentence': 'After allotment, the...","{'id': '35', 'sentence': 'dated 19/20.', 'rhet...","{'id': '36', 'sentence': '1. 1984, by a furthe...","{'id': '37', 'sentence': 'Similar notices were...","{'id': '38', 'sentence': 'On the above content...","{'id': '39', 'sentence': 'The above plea was r...","{'id': '40', 'sentence': 'It has been further ...",...,"{'id': '66', 'sentence': 'It may be mentioned ...","{'id': '67', 'sentence': 'The respondents exce...","{'id': '68', 'sentence': 'For a better appreci...","{'id': '69', 'sentence': 'I have seen the hous...","{'id': '70', 'sentence': 'I accept the rules o...","{'id': '71', 'sentence': '753 Only on the basi...","{'id': '72', 'sentence': 'In this connection, ...","{'id': '73', 'sentence': 'In this connection, ...","{'id': '74', 'sentence': '), B. Sc. , B. Ed. ,...","{'id': '75', 'sentence': 'From the above, it i...","{'id': '76', 'sentence': 'This is not only the...","{'id': '77', 'sentence': 'So it cannot be said...","{'id': '78', 'sentence': 'In such a circum sta...","{'id': '79', 'sentence': 'More so, the respond...","{'id': '80', 'sentence': '1. 84 are not justif...","{'id': '81', 'sentence': 'It is quite un-under...","{'id': '82', 'sentence': 'The respondents were...","{'id': '83', 'sentence': 'Notwithstanding, the...","{'id': '84', 'sentence': 'One should not loose...","{'id': '85', 'sentence': 'On the contrary, the...","{'id': '86', 'sentence': 'In fact, the respond...","{'id': '87', 'sentence': 'Thus the factual pos...","{'id': '88', 'sentence': 'In the case before u...","{'id': '89', 'sentence': 'When the factual pos...","{'id': '90', 'sentence': 'Therefore, the conte...","{'id': '91', 'sentence': 'Where an author ity ...","{'id': '92', 'sentence': 'This finding, in our...","{'id': '93', 'sentence': 'Even conceding that ...","{'id': '94', 'sentence': 'Thereafter the relat...","{'id': '95', 'sentence': 'In this sphere, they...","{'id': '96', 'sentence': 'There is a line of d...","{'id': '97', 'sentence': 'In view of the autho...","{'id': '98'

In [81]:
test_file= pd.DataFrame(columns=['judge_id', 'id', 'sentence'])
for i in range(105):
  for j in range(50):
    if yz1[i][j] is not None:
      test_file= test_file.append({'judge_id': j+1, 'id': yz1[i][j]['id'], 'sentence': yz1[i][j]['sentence']}, ignore_index=True)

test_file

,judge_id,id,sentence
0,1,1,From the Judgment and Order dated 6. 2. 1986 o...
1,2,1,Appeals from the judgment and order dated June...
2,3,1,"Registration number 4, 9 and 10 of 1985 Shanti..."
3,4,1,(Under Article 32 of the Constitution of India).
4,5,1,"305,306 & 307 of 1988."
...,...,...,...
5061,18,105,C. A. numbers 893 to 898/66 dismissed.
5062,21,105,"On the facts and circumstances of the case, th..."
5063,41,105,(1) all india reporter 1939 Lah. 210.
5064,42,105,N. V. K. Appeal dismissed.


In [82]:
# Remove Excess [Size:: ] from EoS
def remove_size(text):
	result = re.search("\[size", text)
	if result is not None:
		text = text[:result.start()]
	return text.strip()

months = ['january', 'jan', 'feb', 'mar', 'apr', 'aug', 'sept', 'oct', 'nov', 'dec', 'feburary', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'on']
def remove_months(text):
    text = text.split()
    text = [word for word in text if word.lower() not in set(months)]
    text = ' '.join(text)
    return text

def remove_nums(text):
    result = re.sub(r'[0-9+]', '', text)
    return result

def remove_comma(text):
    result = re.sub(r',', '', text)
    return result

In [84]:
test_file['rel_sum']=test_file['sentence']
import sys
text = []
lem_text = []
for i in test_file.index:
    test_file['sentence'][i] = remove_size(test_file['sentence'][i])
    test_file['sentence'][i] = remove_months(test_file['sentence'][i])
    test_file['sentence'][i] = remove_nums(test_file['sentence'][i])
    test_file['sentence'][i] = remove_comma(test_file['sentence'][i])
    temp = test_file['sentence'][i].lower().split()
    clean = [word for word in temp if word not in stopwords.words('english')]
    clean = " ".join(clean)
    text.append(clean)
    lem_text.append(clean)

In [85]:
for i in range(len(text)):
    text[i] = text[i].split()

for i in range(len(lem_text)):
    lem_text[i] = lem_text[i].split()

from nltk import PorterStemmer
st = PorterStemmer()
for i in range(len(text)):
    text[i] = [st.stem(word) for word in text[i]]

from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

for i in range(len(lem_text)):
    lem_text[i] = [lemmatizer.lemmatize(word) for word in lem_text[i]]

for i in range(len(text)):
    text[i] = " ".join(text[i])

text[0]

for i in range(len(lem_text)):
    lem_text[i] = " ".join(lem_text[i])

lem_text[0]

'judgment order dated . . alla habad high court civil misc.'

In [86]:
test_file['lem_text'] = np.array(lem_text)
test_file['stem_text'] = np.array(text)

test_file

,judge_id,id,sentence,rel_sum,lem_text,stem_text
0,1,1,From the Judgment and Order dated . . of the ...,From the Judgment and Order dated 6. 2. 1986 o...,judgment order dated . . alla habad high court...,judgment order date . . alla habad high court ...
1,2,1,Appeals from the judgment and order dated of...,Appeals from the judgment and order dated June...,appeal judgment order dated calcutta high cour...,appeal judgment order date calcutta high court...
2,3,1,Registration number and of Shanti Bhushan ...,"Registration number 4, 9 and 10 of 1985 Shanti...",registration number shanti bhushan k. r. nagar...,registr number shanti bhushan k. r. nagaraja r...
3,4,1,(Under Article of the Constitution of India).,(Under Article 32 of the Constitution of India).,(under article constitution india).,(under articl constitut india).
4,5,1,& of .,"305,306 & 307 of 1988.",& .,& .
...,...,...,...,...,...,...
5061,18,105,C. A. numbers to / dismissed.,C. A. numbers 893 to 898/66 dismissed.,c. a. number / dismissed.,c. a. number / dismissed.
5062,21,105,the facts and circumstances of the case there ...,"On the facts and circumstances of the case, th...",fact circumstance case order cost incurred far...,fact circumst case order cost incur far. r. s....
5063,41,105,() all india reporter Lah. .,(1) all india reporter 1939 Lah. 210.,() india reporter lah. .,() india report lah. .
5064,42,105,N. V. K. Appeal dismissed.,N. V. K. Appeal dismissed.,n. v. k. appeal dismissed.,n. v. k. appeal dismissed.


In [88]:
#classifier_model.load_weights('./BERT-Model.h5')
y_preds = np.argmax(classifier_model.predict(test_file.lem_text), axis=1)

In [90]:
len(y_preds)

5066

In [91]:
test_file['relevance']=y_preds

In [93]:
summary_file= pd.DataFrame(columns=['judge_id', 'summary'])
for i in range(50):
      summary_file= summary_file.append({'judge_id': i+1,  'summary': ''}, ignore_index=True)


In [96]:
for i in test_file.index:
  if test_file['relevance'][i]:
    summary_file['summary'][test_file['judge_id'][i]-1]+=(test_file['rel_sum'][i])

summary_file

,judge_id,summary
0,1,"as 'BDA'), the first appellant was constituted..."
1,2,That was a Reference under section 66(1) of th...
2,3,In these appeals under special leave the only ...
3,4,"These two petitions under Art, of the Constitu..."
4,5,A common questions arising for consideration i...
5,6,"The assessee, Elgin Mills limited , at the rel..."
6,7,Basis of claim is glaring disparity between th...
7,8,This appeal by certificate is directed against...
8,9,The respondent carries on the business of fabr...
9,10,The Judgment of the Court was delivered by GAJ...
